# Prueba Técnica Desarrollador IA - Alianza Team


## **Introducción**
> 🎯 **Objetivo**: Desarrollar un modelo predictivo utilizando técnicas de machine learning (ML) para estimar la densidad de licencias comerciales por habitante en cada código postal de la ciudad de Nueva York.

## **Insumos**
| Archivo | Descripción | Fuente |
| :--- | :--- | :--- |
| `demographic-statistics-by-zip-code-1.csv` | La tabla muestra **estadísticas demográficas** de personas que participan en programas financiados por el Departamento de Desarrollo Comunitario y Juvenil de Nueva York (DYCD), organizadas por código postal. | [Demographic Statistics By Zip Code \| data.world](https://data.world/city-of-ny/kku6-nxdu) |
| `DCA_Legally_Operating_Businesses_03062015.xlsx` | Este conjunto de datos incluye licencias emitidas por el Departamento de Protección al Consumidor y al Trabajador (DCWP) a **empresas** (*Premises*) y **personas naturales** (*Individuals*) para que puedan operar legalmente en la ciudad de Nueva York. | [Legally operating businesses \| NYC Open Data](https://data.cityofnewyork.us/Business/Legally-Operating-Businesses/w7w3-xahh/about_data) |

## **Definiciones**
### **Densidad de Licencias Comerciales**
$$ \text{Densidad de Licencias Comerciales} = \frac{\text{Número de Licencias Comerciales}}{\text{Población Total}} $$

## **1. Preparación de los datos**

In [77]:
# Importar librerías (pandas, numpy, matplotlib)
import pandas as pd

In [78]:
# Importar datos
demographics = pd.read_csv('Sources/demographic-statistics-by-zip-code-1.csv')
licenses = pd.read_excel('Sources/DCA_Legally_Operating_Businesses_03062015.xlsx')

In [79]:
# Dimensiones de los datos
print('Shape de los datos de Demografía: ', demographics.shape)
print('Shape de los datos de Licencias: ', licenses.shape)


Shape de los datos de Demografía:  (236, 46)
Shape de los datos de Licencias:  (65798, 14)


### **Tabla 1. Demografía**
Considerando el objetivo de este estudio, las **variables relevantes** para la preparación de los datos son:
- `JURISDICTION NAME`: Código postal.
- `COUNT PARTICIPANTS`: Número de participantes.

In [80]:
demographics.head()

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,10001,44,22,0.50,22,0.50,0,0,44,100,...,44,100,20,0.45,24,0.55,0,0,44,100
1,10002,35,19,0.54,16,0.46,0,0,35,100,...,35,100,2,0.06,33,0.94,0,0,35,100
2,10003,1,1,1.00,0,0.00,0,0,1,100,...,1,100,0,0.00,1,1.00,0,0,1,100
3,10004,0,0,0.00,0,0.00,0,0,0,0,...,0,0,0,0.00,0,0.00,0,0,0,0
4,10005,2,2,1.00,0,0.00,0,0,2,100,...,2,100,0,0.00,2,1.00,0,0,2,100


- Como se puede observar, en la tabla `demographics` hay códigos postales con un recuento de participantes igual a cero.
- Dada la definición de la densidad de licencias comerciales, estos códigos postales no aportarán información relevante al modelo predictivo. Por lo tanto, se procederá a **eliminar** estos registros.

In [81]:
print("Registros por eliminar: ", demographics[demographics['COUNT PARTICIPANTS'] == 0].shape[0])
print("Registros útiles restantes: ", demographics.shape[0] - demographics[demographics['COUNT PARTICIPANTS'] == 0].shape[0])

Registros por eliminar:  131
Registros útiles restantes:  105


In [82]:
# Eliminar registros con 0 participantes
demographics = demographics[demographics['COUNT PARTICIPANTS'] != 0]

Después de eliminar registros innecesarios, podemos revisar las columnas para eliminar aquellas que no aportan información relevante al modelo predictivo.

In [83]:
#Hacer un describe de los datos
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105 entries, 0 to 233
Data columns (total 46 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   JURISDICTION NAME                    105 non-null    int64  
 1   COUNT PARTICIPANTS                   105 non-null    int64  
 2   COUNT FEMALE                         105 non-null    int64  
 3   PERCENT FEMALE                       105 non-null    float64
 4   COUNT MALE                           105 non-null    int64  
 5   PERCENT MALE                         105 non-null    float64
 6   COUNT GENDER UNKNOWN                 105 non-null    int64  
 7   PERCENT GENDER UNKNOWN               105 non-null    int64  
 8   COUNT GENDER TOTAL                   105 non-null    int64  
 9   PERCENT GENDER TOTAL                 105 non-null    int64  
 10  COUNT PACIFIC ISLANDER               105 non-null    int64  
 11  PERCENT PACIFIC ISLANDER             

Por ejemplo:
- Las columnas de censo no aportan información relevante para el modelo predictivo.
- Podemos aprovechar los porcentajes y presindir de las columnas de recuento.
- Las columnas de totales no aportan información relevante para el modelo predictivo.

In [84]:
# Eliminar columnas con 0, COUNT y TOTAL
demographics = demographics.loc[:, (demographics != 0).any(axis=0)]
demographics = demographics[demographics.columns.drop(list(demographics.filter(regex='COUNT')))]
demographics = demographics[demographics.columns.drop(list(demographics.filter(regex='TOTAL')))]

Como resultado pasamos de 45 columnas a 13 columnas de posibles predictores para el modelo.

In [85]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105 entries, 0 to 233
Data columns (total 16 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   JURISDICTION NAME                    105 non-null    int64  
 1   PERCENT FEMALE                       105 non-null    float64
 2   PERCENT MALE                         105 non-null    float64
 3   PERCENT PACIFIC ISLANDER             105 non-null    float64
 4   PERCENT HISPANIC LATINO              105 non-null    float64
 5   PERCENT AMERICAN INDIAN              105 non-null    float64
 6   PERCENT ASIAN NON HISPANIC           105 non-null    float64
 7   PERCENT WHITE NON HISPANIC           105 non-null    float64
 8   PERCENT BLACK NON HISPANIC           105 non-null    float64
 9   PERCENT OTHER ETHNICITY              105 non-null    float64
 10  PERCENT ETHNICITY UNKNOWN            105 non-null    float64
 11  PERCENT PERMANENT RESIDENT ALIEN     